In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [2]:
from my_utils import Dictionary

n_unique = 10

src_dict = Dictionary(['<EOS>'])
tgt_dict = Dictionary(['<BOS>', '<EOS>'])
for n in range(n_unique):
    src_dict.add_word(str(n))
    tgt_dict.add_word(str(n))

In [3]:
from my_utils.toy_data import invert_seq
train = invert_seq(5000, n_unique=n_unique)
test = invert_seq(100, n_unique=n_unique)

In [4]:
import torch
from my_utils import DataLoader
from torch_models.utils import seq2seq

def numericalize(dataset, src_dict, tgt_dict):
    numericalized = [([src_dict(s) for s in src], [tgt_dict(t) for t in tgt]) for src, tgt in dataset]
    return numericalized

# device = 'cuda:0'
device = torch.device('cpu')
trans_func = seq2seq(device)

train_loader = DataLoader(numericalize(train, src_dict, tgt_dict), batch_size=50, trans_func=trans_func)
test_loader = DataLoader(numericalize(test, src_dict, tgt_dict), batch_size=50, trans_func=trans_func)

In [5]:
from torch_models import AttnSeq2Seq

embed_size=64
dropout = 0
model = AttnSeq2Seq(embed_size=embed_size, hidden_size=embed_size, src_vocab_size=len(src_dict), tgt_vocab_size=len(tgt_dict),
                    src_EOS=src_dict('<EOS>'), tgt_BOS=tgt_dict('<BOS>'), tgt_EOS=tgt_dict('<EOS>'),
                    num_layers=2, bidirectional=False, dropout=dropout, rnn='LSTM')
print(model)

AttnSeq2Seq(
  (encoder): RNNEncoder(
    (embedding): Embedding(12, 64, padding_idx=11)
    (rnn): LSTM(64, 64, num_layers=2)
  )
  (decoder): RNNEncoder(
    (embedding): Embedding(13, 64, padding_idx=12)
    (rnn): LSTM(64, 64, num_layers=2)
  )
  (generator): MLP(
    (fc_out): Linear(in_features=64, out_features=12, bias=True)
    (dropout): Dropout(p=0)
    (criterion): CrossEntropyLoss()
    (activation): Tanh()
  )
  (attn_hidden): Linear(in_features=128, out_features=64, bias=True)
  (attention): DotAttn()
)


In [7]:
%%time
from my_utils import Trainer, EvaluatorSeq, EvaluatorLoss
from torch.optim import Adam, SGD

optimizer = Adam(model.parameters())
evaluator = EvaluatorSeq(model, test_loader, measure='BLEU')
# evaluator = EvaluatorLoss(model, test_loader)

trainer = Trainer(model, train_loader)
trainer.train_epoch(optimizer, max_epoch=1,
              evaluator=evaluator, score_monitor=None)

[2018-10-15 15:05:57,519 INFO] epoch [1/1]	loss: 1.0744878113269807	
[2018-10-15 15:05:57,591 INFO] Evaluator BLEU: 0.4189585698166111	


CPU times: user 9.49 s, sys: 154 ms, total: 9.64 s
Wall time: 3.64 s


In [19]:
iter(train_loader)
l = 10
inputs, targets = next(train_loader)
inputs = inputs[:l]
targets = targets[:l]
generated = model.predict(inputs)
print('======= input ======')
for seq in inputs:
    print([src_dict[s.item()] for s in seq])
print('======= output ======')
for seq in generated[:l]:
    print([tgt_dict[s] for s in seq])

======= input ======
['1', '5', '5', '6', '6']
['3', '3', '2', '9']
['5', '2', '8', '0']
['7', '6', '0']
['2', '8', '9']
['2', '7', '0', '2', '8']
['2', '9', '3', '1']
['9', '4', '9']
['8', '0', '8', '1']
['1', '1', '1', '8', '5']
======= output ======
['6', '5', '5', '1']
['9', '2', '3', '3']
['0', '8', '2', '5']
['0', '6', '0']
['9', '8', '2']
['2', '2', '2', '2']
['1', '3', '9', '2']
['9', '4', '9']
['1', '8', '8']
['5', '1', '1', '1']
